# Hotel Rating Prediction
In this project we will inspect the question   
whether we can or can not predict the overall rating on a given hotel  
according to different parameters which we already know

## Data Acquisition 

At this step we are going to acquire the data we need from the website  
and create the DataFrame which will serve us throughout the project.

In [1]:
# First lets import the libraries we need :
import requests
import pandas as pd
import numpy as np
import scipy as sc
from bs4 import BeautifulSoup
import time
import os
import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# The next command will be responsible to display graphs in our notebook.
%matplotlib inline

### Crawling 

We will now fetch what we need from the website "Booking.com"

In [48]:
# creating the function to load a soup object 
def load_soup_object(link):
    response_link = requests.get(link ,headers=0)
    soup_object = BeautifulSoup(response_link.content, "html.parser")
    return soup_object

# Lets get the URL and parser it before we create our DataFrame
URL = "https://www.booking.com/he"
soup = load_soup_object(URL)

# Now we will create the lists that will serve as the columns for the DataFrame
Hotel_Name = []
Staff_Rating = []
Facilities_Rating = []
Cleanliness_Rating = []
Comfort_Rating = []
Value_For_Money_Rating = []
Location_Rating = []
Free_WiFi_Rating = []



# accessing the class that contains the list of regions - not only in israel 
# and creating a list of known cities to be extracted 
# counter will be used to save the index of the desired regions - then to specify the index withing the 'searches' list
searches = soup.find(class_="ia_section-container clearfix wcu-seo-regions-poi").find_all('a')
cities = ["אזור ירושלים","אזור המרכז","גליל","גליל עליון","כנרת","אזור תל אביב","דרום הארץ","ים המלח","נגב","צפון הארץ","רמת הגולן"]
names = []
indexes = []
href_list = []
counter = 0
# now, 'searches' is a list with all the regions,we want to loop through and extract only the ones we want
# we are first getting the name of each object in the list, and checking if it exists in our desired regeions list 'cities'
# if term is met, we are appending its index(integer) to the indexes list
# and extracting the corresponding link with the "href" tag and concatanating it to the original URL
for i in searches:
    name1 = i.get_text().replace("\n", "") 
    if name1 in cities:
        names.append(name1)
        indexes.append(counter)
        href = searches[counter].get("href").strip()
        URL2 = URL+href
        soup2 = load_soup_object(URL2)
        
        # now lets parse it so we can inspect and work with each desired page 
        # each region`s page has a specific "Watch all" link that needs to be accessed
        # therefore, we are accessing the class that contains the only object(link) we need
        # and concatanating the link to the original URL
        
        searches2 = soup2.find(class_="lp-bui-section bui-spacer--largest x2").find_all("a")
        print("counter is : ", counter)
        print("region is : ", name1)
        nexthref = URL + searches2[0].get("href").strip()
        print("full name : ", searches2[0].get_text())
        href_list.append(nexthref)
        print("\nhref is :\n",nexthref, "\n")
    counter+=1

# now after this for loop we have a list that contains all the links to the full page of each desired region

print(names)
print(indexes)
print(names[0])

print("hi")
print(Hotel_Name)
print("hi")
#print(href_list)


    
    

counter is :  1
region is :  נגב
full name :  

צפו בכל 737 המלונות ב-נגב



href is :
 https://www.booking.com/he/searchresults.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4ApCy75MGwAIB0gIkZDlhMDAyZGYtY2IzZC00OTA1LWE0ODMtNWQ5MWNlYzY3YjE42AIG4AIB&sid=8de8809ee157172fa7528d250e3456ef&region=3645& 

counter is :  2
region is :  אזור ירושלים
full name :  

צפו בכל 579 המלונות ב-אזור ירושלים



href is :
 https://www.booking.com/he/searchresults.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4ApCy75MGwAIB0gIkZDlhMDAyZGYtY2IzZC00OTA1LWE0ODMtNWQ5MWNlYzY3YjE42AIG4AIB;sid=aaea3b839d14cee385acc31e38f3dbb6;region=3637& 

counter is :  3
region is :  צפון הארץ
full name :  

צפו בכל 1,509 המלונות ב-צפון הארץ



href is :
 https://www.booking.com/he/searchresults.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4ApCy75MGwAIB0gIkZDlhMDAyZGYtY2IzZC00OTA1LWE0ODMtNWQ5MWNlYzY3YjE42AIG4AIB;sid=dfa1d409fe8

Now after we have a list of the correct and desired regions,  
we are going to access each region's page and loop through all other pages  
so we can extract the information we need regarding each item(hotel) in the page.

In [49]:
#seleniuim variables :
#PATH = "./chromedriver.exe"
#s=Service(PATH)
#driver = webdriver.Chrome(service=s)
# now that we have all the links, we need to parse each one with html.parser so we can work with it
name_index = 0
print("hi")
counter2 = 0
for link in href_list: # i.e north, south, negev
    #driver.get(link)    
    city_soup = load_soup_object(link)
    number_of_pages = int(city_soup.find(class_="d493c719bc").find_all('li')[-1].get_text())
    hotels = city_soup.find(class_="d4924c9e74").find_all('h3')
    print(len(hotels))
    hotel_links = []
    # now we start extracting information for the dataframe. we start by taking the hotel names into the hotel name list
    for item in hotels: # looping through the links for the hotels
        print("hi2")
        print("hotel : ",item.a.find(class_="fcab3ed991 a23c043802").get_text())
        Hotel_Name.append(item.a.find(class_="fcab3ed991 a23c043802").get_text())
        Hotel_Set = set(Hotel_Name)
        Hotel_Name = list(Hotel_Set)
        print("link is :",item.a.get("href").strip())   
        hotel_links.append(item.a.get("href").strip())   
    for hotel in hotel_links: # looping for each hotel link
        print("hi3")
        hotel_soup= load_soup_object(hotel)
        hotel_props= hotel_soup.find(class_="v2_review-scores__body v2_review-scores__body--compared_to_average").find_all('li')
        Staff_Rating.append(hotel_props[0].find(class_="c-score-bar__score").get_text())
        Facilities_Rating.append(hotel_props[1].find(class_="c-score-bar__score").get_text())
        Cleanliness_Rating.append(hotel_props[2].find(class_="c-score-bar__score").get_text())
        Comfort_Rating.append(hotel_props[3].find(class_="c-score-bar__score").get_text())
        Value_For_Money_Rating.append(hotel_props[4].find(class_="c-score-bar__score").get_text())
        Location_Rating.append(hotel_props[5].find(class_="c-score-bar__score").get_text())
        Free_WiFi_Rating.append(hotel_props[6].find(class_="c-score-bar__score").get_text())
    print("Staff_Rating",len(Staff_Rating),"\n",Staff_Rating)
    print("Facilities_Rating",len(Facilities_Rating),"\n",Facilities_Rating)
    print("Cleanliness_Rating",len(Cleanliness_Rating),"\n",Cleanliness_Rating)
    print("Comfort_Rating",len(Comfort_Rating),"\n",Comfort_Rating)
    print("Value_For_Money_Rating",len(Value_For_Money_Rating),"\n",Value_For_Money_Rating)
    print("Location_Rating",len(Location_Rating),"\n",Location_Rating)
    print("Free_WiFi_Rating",len(Free_WiFi_Rating),"\n",Free_WiFi_Rating)
    print("Hotel_Names:", len(Hotel_Name),"\n",Hotel_Name)
    
    dfdict = {"Staff_Rating":Staff_Rating, "Facilities_Rating":Facilities_Rating, "Cleanliness_Rating":Cleanliness_Rating, "Comfort_Rating":Comfort_Rating, "Value_For_Money_Rating":Value_For_Money_Rating, "Location_Rating":Location_Rating, "Free_WiFi_Rating":Free_WiFi_Rating,"Hotel_Names":Hotel_Name}
    df = pd.DataFrame(data=dfdict)
    df.style.set_properties(**{'text-align': 'left'})
    df.style
    print(df)
    break
        
    #for li in hotel_props: # looping inside all "li" section which will be : "staff", "food", "facilities"
        #Staff_Rating.append(li.find(class_="c-score-bar__score").get_text())
#print("CHECKKKKKK\n",hotel_props)


#print(hotel_links)
#for j in pages_links:
        
#print("number of pages in ", names[name_index], " is: " , number_of_pages)
#name_index += 1
#for i in range(number_of_pages):'''
        
        
    

hi
25
hi2
hotel :  נוף מדבר
link is : https://www.booking.com/hotel/il/desert-view.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4ApCy75MGwAIB0gIkZDlhMDAyZGYtY2IzZC00OTA1LWE0ODMtNWQ5MWNlYzY3YjE42AIG4AIB&sid=8de8809ee157172fa7528d250e3456ef&aid=304142&ucfs=1&arphpl=1&dest_id=3645&dest_type=region&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=9d4c6e6faef402e4&srepoch=1652283745&from=searchresults#hotelTmpl
hi2
hotel :  ארץ צאלים
link is : https://www.booking.com/hotel/il/eretz-zeelim.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4ApCy75MGwAIB0gIkZDlhMDAyZGYtY2IzZC00OTA1LWE0ODMtNWQ5MWNlYzY3YjE42AIG4AIB&sid=8de8809ee157172fa7528d250e3456ef&aid=304142&ucfs=1&arphpl=1&dest_id=3645&dest_type=region&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=9d4c6e6faef402e4&srepoch=1652283745&from_sustainabl

In [56]:
df.style.set_properties(**{'text-align': 'right'}).set_table_styles([ dict(selector='th', props=[('text-align', 'right')] ) ])
#dfStyler = df.style.set_properties(**{'text-align': 'right'})
#dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'right')])])
display(df)

,Staff_Rating,Facilities_Rating,Cleanliness_Rating,Comfort_Rating,Value_For_Money_Rating,Location_Rating,Free_WiFi_Rating,Hotel_Names
0,9.4,9.0,8.9,9.2,9.0,9.1,9.1,מלון נובה- מלון מרשת אטלס
1,8.4,7.8,7.8,7.7,7.2,8.1,4.6,סאנסט אין
2,9.3,8.0,8.7,8.4,8.4,9.0,8.0,מלון בוטיק צוקים
3,8.3,7.6,7.6,8.1,7.4,8.3,7.3,מלון אמריקנה אילת
4,9.9,8.8,9.2,8.9,9.3,8.7,9.6,Shamballa Ezuz Desert Hospitality
5,9.5,9.1,9.6,9.3,8.9,8.7,10,מלון לאונרדו נגב
6,9.8,9.5,9.6,9.6,9.4,9.3,9.4,אכסניית בית הערבה
7,9.4,9.3,9.7,9.6,9.5,9.5,8.8,מלון פאלמס אילת
8,9.8,8.6,9.2,8.9,9.1,9.2,7.5,אירוח מדברי קיבוץ לוטן
9,8.5,7.6,8.0,7.9,8.4,8.0,8.0,נוף מדבר
